モジュールをインポート

In [611]:
import random
import time

### ネイピア数
$$e = \lim_{{x \to \infty}} \left(1 + \frac{1}{x}\right)^x$$

In [612]:
def napiers_logarithm(x):
    return (1 + 1 / x) ** x
napier_number = napiers_logarithm(100000000)  # e

### シグモイド関数
$$Sigmoid(x) = \frac{1}{1 + e^{-x}}$$

In [613]:
def sigmoid(x):
    return 1 / (1 + napier_number ** -x)

### シグモイド関数の微分
$$Sigmoid'(x) = Sigmoid(x) \cdot (1 - Sigmoid(x))$$

In [614]:
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

### ReLU関数
$$ReLU(x) = \max(0, x)$$

In [615]:
def relu(x):
    return max(0, x)

### ReLU関数の微分
$$ReLU'(x) = \begin{cases} 
1 & (x > 0) \\
0 & (x ≤ 0)
\end{cases}$$

In [616]:
def relu_derivative(x):
    return 1 if x > 0 else 0

### 自然対数
1. **スケーリング**  
   自然対数の性質 $\ln(a \cdot b) = \ln(a) + \ln(b)$ を利用して、$x$ を 1 に近い値に変換

   $$
   k = 0, \quad x > 2 \text{ の間 } x = \frac{x}{2}, \, k += 1
   $$

   $$
   x < 0.5 \text{ の間 } x = 2 \cdot x, \, k -= 1
   $$

   変換後、$x \in [0.5, 2]$ に収まる

2. **ニュートン法**  
   方程式 $f(y) = e^y - x = 0$ を解くため、ニュートン法を適用

   $$
   y_{n+1} = y_n - \frac{e^{y_n} - x}{e^{y_n}}
   $$

   初期値として $y_0 = x - 1$ を用いる。

3. **結果**  
   最終的な計算：

   $$
   \ln(x) = y + k \cdot \ln(2)
   $$

   $\ln(2) \approx 0.6931471805599453$ を用いる

In [617]:
def ln(x, max_iter=20, tol=1e-12):
    if x <= 0: raise ValueError("x must be positive")
    k = 0
    while x > 2:
        x /= 2
        k += 1
    while x < 0.5:
        x *= 2
        k -= 1
    y = x - 1  # ln(1) = 0 付近の値から開始
    for _ in range(max_iter):
        prev_y = y
        y -= (2.718281828459045**y - x) / (2.718281828459045**y)  # f(y) / f'(y)
        if abs(y - prev_y) < tol:
            break
    return y + k * 0.6931471805599453  # ln(2) ≈ 0.693147

### クロスエントロピー損失
$$ L = -\sum_{i=1}^{N} y_i \cdot \ln(\hat{y}_i + \epsilon)$$

In [618]:
def cross_entropy_loss(y_true, y_pred):
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return -sum([t * ln(p + 1e-9) for t, p in zip(y_true, y_pred)])

### 平方根を計算

In [619]:
def sqrt(x):
    tolerance = 1e-10  # 許容誤差
    estimate = x / 2.0  # 初期推定値
    while True:
        new_estimate = (estimate + x / estimate) / 2  # ニュートン法による更新
        if abs(new_estimate - estimate) < tolerance:  # 収束したら終了
            return new_estimate
        estimate = new_estimate  # 推定値を更新

### ニューラルネットワークを初期化

In [620]:
def initialize_weights(layer_sizes):
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        limit = sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))  # 重みの初期化に使う乱数の範囲
        weights.append([[random.uniform(-limit, limit) for _ in range(layer_sizes[i])] for _ in range(layer_sizes[i+1])])  # 重みは -limit から limit の間の乱数で初期化
        biases.append([0 for _ in range(layer_sizes[i+1])])  # バイアスは0で初期化
    return weights, biases

### 順伝播処理

In [621]:
def forward_propagation(inputs, weights, biases):  # 順伝播処理
    activations = [inputs]
    for W, b in zip(weights, biases):
        z = [
            sum([activations[-1][i] * W[j][i] for i in range(len(activations[-1]))]) + b[j]
            for j in range(len(b))
        ]
        if W != weights[-1]:
            activations.append([relu(z_i) for z_i in z])
        else:
            activations.append([sigmoid(z_i) for z_i in z])
    return activations

### 逆伝播処理

In [622]:
def backward_propagation(activations, y_true, weights, biases, learning_rate):  # 逆伝播処理
    output_layer = activations[-1]
    errors = [
        (output_layer[i] - y_true[i]) * sigmoid_derivative(output_layer[i])
        for i in range(len(y_true))
    ]
    deltas = [errors]
    # 隠れ層の誤差を計算
    for l in range(len(weights)-1, 0, -1):
        hidden_errors = [
            sum([deltas[0][k] * weights[l][k][j] for k in range(len(deltas[0]))]) * relu_derivative(activations[l][j])
            for j in range(len(activations[l]))
        ]
        deltas.insert(0, hidden_errors)
    # 重みとバイアスを更新
    for l in range(len(weights)):
        for i in range(len(weights[l])):
            for j in range(len(weights[l][i])):
                weights[l][i][j] -= learning_rate * deltas[l][i] * activations[l][j]
            biases[l][i] -= learning_rate * deltas[l][i]

    return weights, biases

### 学習

In [623]:
def train(X, y, layer_sizes, epochs, learning_rate):  # 学習
    weights, biases = initialize_weights(layer_sizes)
    start = time.time()
    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(X)):
            activations = forward_propagation(X[i], weights, biases)
            total_loss += cross_entropy_loss(y[i], activations[-1])
            weights, biases = backward_propagation(activations, y[i], weights, biases, learning_rate)
        m = epoch // (epochs // 20) + 1
        print(f"\rEpoch {epoch+1}/{epochs}, Loss: {total_loss/len(X):.10f}, [{'+'*m}{' '*(20-m)}]{' '*5}",end="")
    print(f"Training time: {time.time()-start:.2f} seconds")
    return weights, biases

In [624]:
def predict(X, weights, biases):  # 予測
    outputs = []
    for i in range(len(X)):  # Prediction
        outputs.append(forward_propagation(X[i], weights, biases)[-1])
    return outputs

In [625]:
def accuracy(predict, y):  # 予測精度の計算
    accuracy = 0
    for i in range(len(predict)):  # Prediction
        accuracy += 1 if [0 if predict[i][0]<0.5 else 1] == y[i] else 0
    return accuracy / len(predict)

### メインコード

In [626]:
# DataSet for XOR
X = [[0, 0], [0, 1], [1, 0], [1, 1]]  # Input
y = [[0], [1], [1], [0]]  # Output

epochs = 5000  # Number of epochs
learning_rate = 0.01  # Learning rate
layer_sizes = [2, 8, 16, 8, 1]  # 2 input -> 8 hidden -> 16 hidden -> 8 hidden -> 1 output

weights, biases = train(X, y, layer_sizes, epochs, learning_rate)

predict_y = predict(X, weights, biases)
print(predict_y)
for i in range(len(X)):
    print(f"Input: {X[i]}, Answer: {y[i]}, Predict: {[0 if predict_y[i][0]<0.5 else 1]}, Predict_num: {predict_y[i]}")

accuracy_num = accuracy(predict_y, y)
print(f"Accuracy: {accuracy_num*100:.1f}%")

Epoch 5000/5000, Loss: 0.0012611034, [++++++++++++++++++++]     Training time: 2.91 seconds
[[0.0038777671013214584], [0.9968500645134346], [0.9981157718857484], [0.0013309809580459217]]
Input: [0, 0], Answer: [0], Predict: [0], Predict_num: [0.0038777671013214584]
Input: [0, 1], Answer: [1], Predict: [1], Predict_num: [0.9968500645134346]
Input: [1, 0], Answer: [1], Predict: [1], Predict_num: [0.9981157718857484]
Input: [1, 1], Answer: [0], Predict: [0], Predict_num: [0.0013309809580459217]
Accuracy: 100.0%
